In [1]:
import pandas as pd

books = pd.read_csv("books_with_categories.csv")

In [3]:
from transformers import pipeline

pipe = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k = None)

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu


model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

In [10]:
books["description"][0]
pipe(books["description"][0])


[[{'label': 'fear', 'score': 0.654839813709259},
  {'label': 'neutral', 'score': 0.16985264420509338},
  {'label': 'sadness', 'score': 0.11640956997871399},
  {'label': 'surprise', 'score': 0.020700659602880478},
  {'label': 'disgust', 'score': 0.019100705161690712},
  {'label': 'joy', 'score': 0.015161501243710518},
  {'label': 'anger', 'score': 0.003935148939490318}]]

In [16]:
sentences = (books["description"][0].split("."))

predictions = pipe(sentences)

In [26]:
sentences[3]

' Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist'

In [27]:
predictions[3]

[{'label': 'fear', 'score': 0.9281682372093201},
 {'label': 'anger', 'score': 0.03219093754887581},
 {'label': 'neutral', 'score': 0.012808676809072495},
 {'label': 'sadness', 'score': 0.00875687412917614},
 {'label': 'surprise', 'score': 0.008597913198173046},
 {'label': 'disgust', 'score': 0.00843182671815157},
 {'label': 'joy', 'score': 0.0010455831652507186}]

In [28]:
sorted(predictions[0], key=lambda x:x["label"])

[{'label': 'anger', 'score': 0.009156367741525173},
 {'label': 'disgust', 'score': 0.0026284796185791492},
 {'label': 'fear', 'score': 0.06816233694553375},
 {'label': 'joy', 'score': 0.04794260486960411},
 {'label': 'neutral', 'score': 0.14038586616516113},
 {'label': 'sadness', 'score': 0.0021221658680588007},
 {'label': 'surprise', 'score': 0.7296021580696106}]

In [30]:
import numpy as np

emotion_labels = ["anger","digust","fear","joy","sadness","surprise", "neutral"]
isbn=[]
emotion_scores = {label: [] for label in emotion_labels}


def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key= lambda x:x["label"])
        for index,label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}


In [32]:
for i in range(10):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = pipe(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

In [33]:
emotion_scores

{'anger': [np.float64(0.06413359194993973),
  np.float64(0.6126194596290588),
  np.float64(0.06413359194993973),
  np.float64(0.3514842092990875),
  np.float64(0.08141259849071503),
  np.float64(0.23222461342811584),
  np.float64(0.5381842851638794),
  np.float64(0.06413359194993973),
  np.float64(0.30066993832588196),
  np.float64(0.06413359194993973)],
 'digust': [np.float64(0.2735915184020996),
  np.float64(0.3482845425605774),
  np.float64(0.10400668531656265),
  np.float64(0.15072239935398102),
  np.float64(0.18449528515338898),
  np.float64(0.7271749973297119),
  np.float64(0.15585505962371826),
  np.float64(0.10400668531656265),
  np.float64(0.2794816195964813),
  np.float64(0.17792613804340363)],
 'fear': [np.float64(0.9281682372093201),
  np.float64(0.9425276517868042),
  np.float64(0.9723208546638489),
  np.float64(0.3607059419155121),
  np.float64(0.09504329413175583),
  np.float64(0.05136280506849289),
  np.float64(0.7474273443222046),
  np.float64(0.4044977128505707),
  np

In [36]:
from tqdm import tqdm

emotion_labels = ["anger","digust","fear","joy","sadness","surprise", "neutral"]
isbn=[]
emotion_scores = {label: [] for label in emotion_labels}


for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = pipe(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 5197/5197 [13:28<00:00,  6.43it/s]


In [37]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["isbn13"] = isbn

In [38]:
emotions_df

,anger,digust,fear,joy,sadness,surprise,neutral,isbn13
0,0.064134,0.273592,0.928168,0.932798,0.646216,0.967158,0.729602,9780002005883
1,0.612619,0.348285,0.942528,0.704422,0.887940,0.111690,0.252546,9780002261982
2,0.064134,0.104007,0.972321,0.767238,0.549477,0.111690,0.078765,9780006178736
3,0.351484,0.150722,0.360706,0.251881,0.732685,0.111690,0.078765,9780006280897
4,0.081413,0.184495,0.095043,0.040564,0.884390,0.475881,0.078765,9780006280934
...,...,...,...,...,...,...,...,...
5192,0.148209,0.030643,0.919165,0.255173,0.853721,0.980877,0.030656,9788172235222
5193,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765,9788173031014
5194,0.009997,0.009929,0.339218,0.947779,0.375754,0.066685,0.057625,9788179921623
5195,0.064134,0.104007,0.459269,0.759457,0.951104,0.368111,0.078765,9788185300535


In [39]:
books = pd.merge(books, emotions_df, on="isbn13")

In [40]:
books.to_csv("books_with_emotions.csv", index=False)